In [85]:
import mechanicalsoup as ms
import pandas as pd
import numpy as np
import requests
import re
import os
import glob
import csv
import numpy as np
import math
from bs4 import BeautifulSoup
import urllib.request as rq
import urllib
from zipfile import ZipFile
from io import BytesIO
import lxml
from sklearn.linear_model import LinearRegression
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import RFE

## Function to download the data

In [14]:

def downloadhistoricaldata(trainQ, testQ, tables,session, flag):
    for link in tables:
        if(trainQ in link['href'] or testQ in link['href']):
            c = 'https://freddiemac.embs.com/FLoan/Data/' + link['href']
            response = session.get(c)
            z = ZipFile(BytesIO(response.content)) 
            z.extractall(os.getcwd())
            flag = 1
    return flag

# Function to login into the website

In [13]:
def login(login, password, trainQ, testQ):   
    s = requests.Session()
    url = "https://freddiemac.embs.com/FLoan/secure/auth.php"
    url2 = "https://freddiemac.embs.com/FLoan/Data/download.php"
    browser = ms.Browser(session = s)
    print("Logging in....")
    login_page = browser.get(url)
    login_form = login_page.soup.find("form",{"class":"form"})
    login_form.find("input", {"name":"username"})["value"] = login
    login_form.find("input", {"name":"password"})["value"] = password
    response = browser.submit(login_form, login_page.url)
    login_page2 = browser.get(url2)
    print("To the continue page...")

    next_form = login_page2.soup.find("form",{"class":"fmform"})
    a= next_form.find("input",{"name": "accept"}).attrs
    a['checked']=True

    response2 = browser.submit(next_form, login_page2.url)
    print("Start Downloading from..."+ response2.url)
    table = response2.soup.find("table",{"class":"table1"})
    
    t = table.find_all('a')
    flag = 0
    flag = downloadhistoricaldata(trainQ, testQ, t,s, flag) 

    if flag == 1:
        print("Data downloaded successfully!!")
    else:
        print("Error in downloading data")


In [15]:
login('rishi.r.rajani@gmail.com','jQcQFxI=','Q12005','Q22005')

Logging in....
To the continue page...
Start Downloading from...https://freddiemac.embs.com/FLoan/Data/download.php
Data downloaded successfully!!


## Function to convert downloaded data into CSV

In [3]:
def downloaddataincsv(trainQ, testQ):
    trainfile = "historical_data1_"+ trainQ + ".txt"
    testfile = "historical_data1_"+ testQ + ".txt"
    f1 = "train_" + trainQ + ".csv"
    f2 = "test_" + testQ + ".csv"
    with open(f1, 'w',encoding='utf-8') as file: 
        df = pd.read_csv(trainfile ,delimiter ="|", names=['credit_score','first_payment_date','fthb_flag','matr_date','msa',"mortage_insurance_pct",'no_of_units','occupancy_status','cltv','dti_ratio','original_upb','original_ltv','original_int_rt','channel','ppm_flag','product_type','property_state', 'prop_type','zipcode','loan_seq_number','loan_purpose', 'original_loan_term','number_of_borrowers','sellers_name','servicer_name','super_conforming_flag'],skipinitialspace=True)   
        df.to_csv(file, header=True,index=False, mode='a')
        print("%s csv generated!"%file )
        
    with open(f2, 'w',encoding='utf-8') as file: 
        df = pd.read_csv(testfile ,delimiter ="|", names=['credit_score','first_payment_date','fthb_flag','matr_date','msa',"mortage_insurance_pct",'no_of_units','occupancy_status','cltv','dti_ratio','original_upb','original_ltv','original_int_rt','channel','ppm_flag','product_type','property_state', 'prop_type','zipcode','loan_seq_number','loan_purpose', 'original_loan_term','number_of_borrowers','sellers_name','servicer_name','super_conforming_flag'],skipinitialspace=True)   
        df.to_csv(file, header=True,index=False, mode='a')
        print("%s csv generated!"%file )

## Retreiving Data from Downloaded 

In [4]:
def getTrainTest(trainQ, testQ):
    f1 = "train_" + trainQ + ".csv"
    f2 = "test_" + testQ + ".csv"
    trf = pd.read_csv(f1)
    tsf = pd.read_csv(f2)
    return (trf, tsf)

## Function to clean Data

In [6]:
def cleaningdata(data):
    #Dropping Credit scores above 850 and less than 301
    data=data.drop(data['credit_score'].loc[(data['credit_score'] < 301) | (data['credit_score'] > 850)].index)
    data=data.dropna(subset=['credit_score'])
    data=data.dropna(subset=['first_payment_date'])
    data['fthb_flag'] = data['fthb_flag'].fillna("NA")
    #Dropping not applicable MSA data
    data=data.dropna(subset=['msa'])
    data['mortage_insurance_pct'] = data['mortage_insurance_pct'].fillna(0)
    data['no_of_units'] = data['no_of_units'].fillna(0)
    data['cltv'] = data['cltv'].fillna(0)
    data['dti_ratio'] = data['dti_ratio'].fillna(0)
    data['original_ltv'] = data['original_ltv'].fillna(0)
    data['ppm_flag'] = data['ppm_flag'].fillna("U")
    data['prop_type']=data['prop_type'].fillna('NA')
    data['loan_purpose']=data['loan_purpose'].fillna('NA')
    data = data.dropna(subset=['zipcode'])
    data['number_of_borrowers'] = data['number_of_borrowers'].fillna(1)
    data['super_conforming_flag'] = data['super_conforming_flag'].fillna("N")
    
    return data

## Function to convert Data into Numbers

In [7]:
def convertnumbersdata(data):
    data['fthb_flag'] = data['fthb_flag'].replace(['Y','N','NA'],[1,2,3])
    data['occupancy_status'] = data['occupancy_status'].replace(['I','O','S','P'],[1,2,3,4])
    data['channel'] = data['channel'].replace(['B','C','R','T'],[1,2,3,4])
    data['ppm_flag'] = data['ppm_flag'].replace(['Y','N','U'],[1,2,3])
    data['prop_type'] = data['prop_type'].replace(['CO','LH','PU','MH','SF','CP','99'],[1,2,3,4,5,6,7])
    data['loan_purpose'] = data['loan_purpose'].replace(['P','C','N','NA'],[1,2,3,4])
    data['super_conforming_flag'] = data['super_conforming_flag'].replace(['Y','N'],[0,1])
    
    return data

## Function to Change data type to integer for linear regression 

In [8]:
def changedatatype(data):
    data[['credit_score','msa','no_of_units','mortage_insurance_pct','cltv','dti_ratio','original_ltv','zipcode','number_of_borrowers']]=data[['credit_score','msa','no_of_units','cltv','mortage_insurance_pct','dti_ratio','original_ltv','zipcode','number_of_borrowers']].astype('int64')
    data[['fthb_flag','occupancy_status','channel']] = data[['fthb_flag','occupancy_status','channel']].astype('int64')
    data[['ppm_flag','prop_type','loan_purpose','super_conforming_flag']]= data[['ppm_flag','prop_type','loan_purpose','super_conforming_flag']].astype('int64')
    data[['product_type','property_state']] = data[['product_type','property_state']].astype('str')
    data[['loan_seq_number','sellers_name','servicer_name']] = data[['loan_seq_number','sellers_name','servicer_name']].astype('str')
    return data

In [28]:
def mean_absolute_percentage_error(y_train,y_predicted): 
    y_train, y_predicted = np.array(y_train), np.array(y_predicted)
    return np.mean(np.abs((y_train,y_predicted) / y_train)) * 100
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math 

In [118]:
def linearRegression(x1,y1,x2,y2):
    regressor = LinearRegression()
    regressor.fit(x1,y1)
    y_pred_train = regressor.predict(x1)
    y_pred_test = regressor.predict(x2)
    
    print('\nTraining Data')
    print('Score:',regressor.score(x1,y1))
    MAE = mean_absolute_error(y1,y_pred_train)
    print('MAE of Training Data =', MAE)
    ## Mean squared error
    MSE = mean_squared_error(y1,y_pred_train)
    RMSE = math.sqrt(MSE)
    print('RMSE of Training Data =',RMSE)
    ## R-square score of this model
    R2 = r2_score(y1,y_pred_train)
    print('R2 of Training Data =',R2)
    ## MAPE of this model
    MAPE=mean_absolute_percentage_error(y1,y_pred_train)
    print('MAPE of Training Data =',MAPE,'\n')
    
    print('\nTesting Data')
    print('Score:',regressor.score(x2,y2))
    MAE = mean_absolute_error(y2,y_pred_test)
    print('MAE of Training Data =', MAE)
    ## Mean squared error
    MSE = mean_squared_error(y2,y_pred_test)
    RMSE = math.sqrt(MSE)
    print('RMSE of Training Data =',RMSE)
    ## R-square score of this model
    R2 = r2_score(y2,y_pred_test)
    print('R2 of Training Data =',R2)
    ## MAPE of this model
    MAPE=mean_absolute_percentage_error(y2,y_pred_test)
    print('MAPE of Training Data =',MAPE)
    

In [163]:
from sklearn.ensemble import RandomForestRegressor
def RandomForestRegression(x1,y1,x2,y2):
    rfc = RandomForestRegressor(n_estimators=50,random_state=np.random)
    rfc.fit(x1,y1)
    y_pred_train = rfc.predict(x1)
    y_pred_test = rfc.predict(x2)
    
    print('\nTraining Data')
    print('\nScore',rfc.score(x1,y1))
    MAE = mean_absolute_error(y1,y_pred_train)
    print('MAE of Training Data =', MAE)
    ## Mean squared error
    MSE = mean_squared_error(y1,y_pred_train)
    RMSE = math.sqrt(MSE)
    print('RMSE of Training Data =',RMSE)
    ## R-square score of this model
    R2 = r2_score(y1,y_pred_train)
    print('R2 of Training Data =',R2)
    ## MAPE of this model
    MAPE=mean_absolute_percentage_error(y1,y_pred_train)
    print('MAPE of Training Data =',MAPE)
    
    print('\n Testing Data')
    print('Score',rfc.score(x2,y2))
    MAE = mean_absolute_error(y2,y_pred_test)
    print('MAE of Training Data =', MAE)
    ## Mean squared error
    MSE = mean_squared_error(y2,y_pred_test)
    RMSE = math.sqrt(MSE)
    print('RMSE of Training Data =',RMSE)
    ## R-square score of this model
    R2 = r2_score(y2,y_pred_test)
    print('R2 of Training Data =',R2)
    ## MAPE of this model
    MAPE=mean_absolute_percentage_error(y2,y_pred_test)
    print('MAPE of Training Data =',MAPE)
    

In [122]:
def neuralnetworks(x1,y1,x2,y2):
    neuralNetwork = MLPRegressor(hidden_layer_sizes=(15,15,15))
    neuralNetwork.fit(x1,y1)
    y_pred_train = neuralNetwork.predict(x1)
    y_pred_test = neuralNetwork.predict(x2)
    
    print('\nTraining Data')
    print('Score',neuralNetwork.score(x1,y1))
    MAE = mean_absolute_error(y1,y_pred_train)
    print('MAE of Training Data =', MAE)
    ## Mean squared error
    MSE = mean_squared_error(y1,y_pred_train)
    RMSE = math.sqrt(MSE)
    print('RMSE of Training Data =',RMSE)
    ## R-square score of this model
    R2 = r2_score(y1,y_pred_train)
    print('R2 of Training Data =',R2)
    ## MAPE of this model
    MAPE=mean_absolute_percentage_error(y1,y_pred_train)
    print('MAPE of Training Data =',MAPE)
    
    print('\nTesting Data')
    print('Score: ',neuralNetwork.score(x2,y2))
    MAE = mean_absolute_error(y2,y_pred_test)
    print('MAE of Training Data =', MAE)
    ## Mean squared error
    MSE = mean_squared_error(y2,y_pred_test)
    RMSE = math.sqrt(MSE)
    print('RMSE of Training Data =',RMSE)
    ## R-square score of this model
    R2 = r2_score(y2,y_pred_test)
    print('R2 of Training Data =',R2)
    ## MAPE of this model
    MAPE=mean_absolute_percentage_error(y2,y_pred_test)
    print('MAPE of Training Data =',MAPE)

In [117]:
#login ('a','b','c','d')
trainQ = 'Q12005'
testQ = 'Q22005'
#downloaddataincsv(trainQ,testQ)
train_data,test_data = getTrainTest(trainQ,testQ)
data_train = cleaningdata(train_data)
data_test = cleaningdata(test_data)
data_train = convertnumbersdata(data_train)
data_test = convertnumbersdata(data_test)
data_train = changedatatype(data_train)
data_test = changedatatype(data_test)
x1 = data_train.copy(deep=True)
y1 = data_train.original_int_rt
x1.drop(['loan_seq_number','property_state','sellers_name','servicer_name','super_conforming_flag' ,'product_type', 'original_int_rt','zipcode','msa'], axis =1, inplace = True)
x2 =  data_test.copy(deep =True)
y2 = data_test.original_int_rt
x2.drop(['loan_seq_number','property_state','sellers_name','servicer_name','super_conforming_flag' ,'product_type', 'original_int_rt','zipcode','msa'], axis =1, inplace = True)
linearRegression(x1,y1,x2,y2)


Training Data
Score: 0.38545746359153465
MAE of Training Data = 0.21086584631607977
RMSE of Training Data = 0.28446190667399945
R2 of Training Data = 0.38545746359153465
MAPE of Training Data = 100.12502718722006 


Testing Data
Score: 0.15267794422249192
MAE of Training Data = 0.24428970743232073
RMSE of Training Data = 0.318664522425998
R2 of Training Data = 0.15267794422249192
MAPE of Training Data = 99.18266484244911


In [126]:
RandomForestRegression(x1,y1,x2,y2)


Training Data

Score 0.9218957303180438
MAE of Training Data = 0.07514743491903093
RMSE of Training Data = 0.10141110859760777
R2 of Training Data = 0.9218957303180438
MAPE of Training Data = 100.04384400037725

 Testing Data
Score 0.2081089446503399
MAE of Training Data = 0.23777447041809666
RMSE of Training Data = 0.30806487126844917
R2 of Training Data = 0.2081089446503399
MAPE of Training Data = 100.11708185075616


In [123]:
neuralnetworks(x1,y1,x2,y2)


Training Data
Score -2838.7491458511613
MAE of Training Data = 18.75910829067956
RMSE of Training Data = 19.33695698718669
R2 of Training Data = -2838.7491458511613
MAPE of Training Data = 166.50619658433507

Testing Data
Score:  -3184.5549780756583
MAE of Training Data = 18.987486471990255
RMSE of Training Data = 19.53898556491105
R2 of Training Data = -3184.5549780756583
MAPE of Training Data = 164.76354734501652


In [81]:
import statsmodels.api as sm
def stepwise_selection(X1, y1, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):

    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X1.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y1, sm.add_constant(pd.DataFrame(X1[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y1, sm.add_constant(pd.DataFrame(X1[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [100]:
def perform_recursiveFE(xaxis,yaxis):
    lr = LinearRegression()
    selector = RFE(lr,10)
    feat = selector.fit(xaxis, yaxis)
    prediction=feat.predict(xaxis)
    score=r2_score(yaxis,prediction)
    print("RFE r2 score: ",score)
    rankfeatures=pd.DataFrame(list(zip(xaxis.columns,sorted(feat.ranking_))),columns=["features","ranking"])
    print(rankfeatures)
    lis = rankfeatures.loc[rankfeatures['ranking'] == 1]
    return lis['features'].values

In [151]:
x1 = data_train.copy(deep = True)
y1 = x1.original_int_rt
x1 . drop(['loan_seq_number','product_type','property_state','sellers_name','servicer_name', 'original_int_rt',], axis =1 , inplace = True)

In [152]:
result = stepwise_selection(x1,y1)
print (result)

C:\Users\rishi\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  credit_score                   with p-value 0.0
Add  original_ltv                   with p-value 0.0
Add  original_upb                   with p-value 0.0
Add  original_loan_term             with p-value 0.0
Add  cltv                           with p-value 0.0
Add  matr_date                      with p-value 0.0
Add  super_conforming_flag          with p-value 0.0
Add  loan_purpose                   with p-value 0.0
Add  occupancy_status               with p-value 0.0
Add  first_payment_date             with p-value 0.0
Add  zipcode                        with p-value 2.99006e-114
Add  prop_type                      with p-value 5.72251e-58
Add  ppm_flag                       with p-value 2.52127e-56
Add  channel                        with p-value 2.73204e-56
Add  fthb_flag                      with p-value 2.96895e-65
Add  mortage_insurance_pct          with p-value 3.2769e-40
Add  number_of_borrowers            with p-value 1.58178e-14
Add  no_of_units                    with p-

In [104]:
lis = perform_recursiveFE(x1,y1)
print(lis)

RFE r2 score:  0.33164643323586684
                 features  ranking
0            credit_score        1
1      first_payment_date        1
2               fthb_flag        1
3               matr_date        1
4                     msa        1
5   mortage_insurance_pct        1
6             no_of_units        1
7        occupancy_status        1
8                    cltv        1
9               dti_ratio        1
10           original_upb        2
11           original_ltv        3
12                channel        4
13               ppm_flag        5
14              prop_type        6
15                zipcode        7
16           loan_purpose        8
17     original_loan_term        9
18    number_of_borrowers       10
19  super_conforming_flag       11
['credit_score' 'first_payment_date' 'fthb_flag' 'matr_date' 'msa'
 'mortage_insurance_pct' 'no_of_units' 'occupancy_status' 'cltv'
 'dti_ratio']


In [153]:
stepwise_data_train= x1[result]
rfe_data_train = x1[lis]

In [154]:
x2 = data_test.copy(deep = True)
y2 = x2.original_int_rt
stepwise_data_est = x2[result]
rfe_data = x2[lis]

In [134]:
linearRegression(stepwise_data_train, y1, stepwise_data_est,y2)


Training Data
Score: 0.38608112429318964
MAE of Training Data = 0.21067809832173592
RMSE of Training Data = 0.28431752875494376
R2 of Training Data = 0.38608112429318964
MAPE of Training Data = 100.12489128928084 


Testing Data
Score: 0.1539055593141737
MAE of Training Data = 0.24399943675945607
RMSE of Training Data = 0.31843359537152577
R2 of Training Data = 0.1539055593141737
MAPE of Training Data = 99.18236412891345


In [135]:
linearRegression(rfe_data_train, y1, rfe_data,y2)


Training Data
Score: 0.3381300989541415
MAE of Training Data = 0.22065949208642552
RMSE of Training Data = 0.2952123067163569
R2 of Training Data = 0.3381300989541415
MAPE of Training Data = 100.13487432019272 


Testing Data
Score: 0.09773099128853391
MAE of Training Data = 0.2530221212956229
RMSE of Training Data = 0.3288345788661211
R2 of Training Data = 0.09773099128853391
MAPE of Training Data = 99.21690667658717


In [136]:
RandomForestRegression(stepwise_data_train, y1, stepwise_data_est,y2)


Training Data

Score 0.9259361351947022
MAE of Training Data = 0.07271367267027849
RMSE of Training Data = 0.09875323418340701
R2 of Training Data = 0.9259361351947023
MAPE of Training Data = 100.04543176365874

 Testing Data
Score 0.24566681783679115
MAE of Training Data = 0.23192393213864076
RMSE of Training Data = 0.30067067027943206
R2 of Training Data = 0.24566681783679112
MAPE of Training Data = 100.00593074172043


In [137]:
RandomForestRegression(rfe_data_train, y1, rfe_data,y2)


Training Data

Score 0.9119838188882907
MAE of Training Data = 0.08021662153516103
RMSE of Training Data = 0.10765380959915967
R2 of Training Data = 0.9119838188882907
MAPE of Training Data = 100.04738213954127

 Testing Data
Score 0.1366798150214652
MAE of Training Data = 0.24758598068188836
RMSE of Training Data = 0.32165877763626355
R2 of Training Data = 0.1366798150214652
MAPE of Training Data = 100.47334383270277


In [138]:
neuralnetworks(stepwise_data_train, y1, stepwise_data_est,y2)


Training Data
Score -4186.644459268047
MAE of Training Data = 17.690275176339362
RMSE of Training Data = 23.481889680694145
R2 of Training Data = -4186.644459268047
MAPE of Training Data = 242.6104285171757

Testing Data
Score:  -4579.158159801051
MAE of Training Data = 17.27361913223816
RMSE of Training Data = 23.428781236102928
R2 of Training Data = -4579.158159801051
MAPE of Training Data = 234.50372083185127


In [139]:
neuralnetworks(rfe_data_train, y1, rfe_data,y2)


Training Data
Score -1869.9927407588116
MAE of Training Data = 15.044301628094503
RMSE of Training Data = 15.69582665271754
R2 of Training Data = -1869.9927407588116
MAPE of Training Data = 132.55355503914325

Testing Data
Score:  -2203.6168073367485
MAE of Training Data = 15.737134974816941
RMSE of Training Data = 16.254589894638865
R2 of Training Data = -2203.6168073367485
MAPE of Training Data = 135.96503133998976


In [148]:
quat_train = ['Q12007', 'Q22007', 'Q32007', 'Q42007']
quat_test = ['Q22007', 'Q32007', 'Q42007','Q12008']

In [165]:
for i,j in zip(quat_train,quat_test):
    downloaddataincsv(i,j)
    train_data,test_data = getTrainTest(i,j)
    data_train = cleaningdata(train_data)
    data_test = cleaningdata(test_data)
    data_train = convertnumbersdata(data_train)
    data_test = convertnumbersdata(data_test)
    data_train = changedatatype(data_train)
    data_test = changedatatype(data_test)
    random_data_train = data_train[result] 
    random_data_test = data_test[result]
    y1 = data_train.original_int_rt
    y2 = data_test.original_int_rt
    RandomForestRegression(random_data_train,y1, random_data_test,y2)

<_io.TextIOWrapper name='train_Q12007.csv' mode='w' encoding='utf-8'> csv generated!
<_io.TextIOWrapper name='test_Q22007.csv' mode='w' encoding='utf-8'> csv generated!

Training Data

Score 0.8984367291317561
MAE of Training Data = 0.08441300252031134
RMSE of Training Data = 0.11377538935850015
R2 of Training Data = 0.8984367291317561
MAPE of Training Data = 100.0459044181307

 Testing Data
Score 0.2552613986875537
MAE of Training Data = 0.25548289353121545
RMSE of Training Data = 0.33663139535665854
R2 of Training Data = 0.2552613986875537
MAPE of Training Data = 99.94011028156315
<_io.TextIOWrapper name='train_Q22007.csv' mode='w' encoding='utf-8'> csv generated!
<_io.TextIOWrapper name='test_Q32007.csv' mode='w' encoding='utf-8'> csv generated!

Training Data

Score 0.9084616707062401
MAE of Training Data = 0.08779762735934335
RMSE of Training Data = 0.11801961014691317
R2 of Training Data = 0.9084616707062401
MAPE of Training Data = 100.04930003666144

 Testing Data
Score 0.130490

In [164]:
downloaddataincsv('Q12007','Q22007')
train_data,test_data = getTrainTest(i,j)
data_train = cleaningdata(train_data)
data_test = cleaningdata(test_data)
data_train = convertnumbersdata(data_train)
data_test = convertnumbersdata(data_test)
data_train = changedatatype(data_train)
data_test = changedatatype(data_test)
random_data_train = data_train[result] 
random_data_test = data_test[result]
y1 = data_train.original_int_rt
y2 = data_test.original_int_rt
RandomForestRegression(random_data_train,y1, random_data_test,y2)

<_io.TextIOWrapper name='train_Q12007.csv' mode='w' encoding='utf-8'> csv generated!
<_io.TextIOWrapper name='test_Q22007.csv' mode='w' encoding='utf-8'> csv generated!

Training Data

Score 0.9159519878715316
MAE of Training Data = 0.09595147966506558
RMSE of Training Data = 0.1288807232335447
R2 of Training Data = 0.9159519878715316
MAPE of Training Data = 100.05532171799427

 Testing Data
Score 0.14055029336881752
MAE of Training Data = 0.38346335720753594
RMSE of Training Data = 0.4660539081957259
R2 of Training Data = 0.14055029336881752
MAPE of Training Data = 102.42424866378211
